Human Computer Interaction CS449 – CS549

Assignment 5: Development of Gesture-based interaction using Mediapipe

In [2]:
print("hello")

hello


In [1]:
import cv2
import mediapipe as mp

# MediaPipe için gerekli modülleri başlat
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Kameradan video akışını başlat
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(
        min_detection_confidence=0.5,  # Algılama güven seviyesi
        min_tracking_confidence=0.5   # İzleme güven seviyesi
) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Kamera bulunamadı!")
            break

        # Kameradan alınan görüntüyü RGB formatına dönüştür
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(frame_rgb)

        # Yüz işaret noktalarını çizin
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)

        # Görüntüyü göster
        cv2.imshow("Yüz Jest Algılama", frame)

        # Çıkmak için 'q' tuşuna basın
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

[12/08 01:35:11.222421][info][1815152][Context.cpp:69] Context created with config: default config!
[12/08 01:35:11.222440][info][1815152][Context.cpp:74] Context work_dir=/Users/ulasyildiz/Documents/Gesture-Based Interaction/CS449_Gesture_Intereaction
[12/08 01:35:11.222442][info][1815152][Context.cpp:77] 	- SDK version: 1.9.4
[12/08 01:35:11.222443][info][1815152][Context.cpp:78] 	- SDK stage version: main
[12/08 01:35:11.222445][info][1815152][Context.cpp:82] get config EnumerateNetDevice:false
[12/08 01:35:11.222447][info][1815152][MacPal.cpp:36] createObPal: create MacPal!
[12/08 01:35:11.222793][info][1815152][MacPal.cpp:104] Create PollingDeviceWatcher!
[12/08 01:35:11.222800][info][1815152][DeviceManager.cpp:15] Current found device(s): (0)
[12/08 01:35:11.222802][info][1815152][Pipeline.cpp:15] Try to create pipeline with default device.
[12/08 01:35:11.222803][warning][1815152][ObException.cpp:5] No device found, fail to create pipeline!
[12/08 01:35:11.223163][info][1815152]

OpenCV: not authorized to capture video (status 0), requesting...
OpenCV: camera failed to properly initialize!
[ WARN:0@16.510] global cap.cpp:323 open VIDEOIO(OBSENSOR): raised unknown C++ exception!


I0000 00:00:1733610911.301855 1815152 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733610911.304772 1815615 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733610911.307410 1815615 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [2]:
import numpy as np

def calculate_eye_aspect_ratio(eye_landmarks):
    # Göz noktalarını kullanarak EAR hesapla
    A = np.linalg.norm(np.array(eye_landmarks[1]) - np.array(eye_landmarks[5]))
    B = np.linalg.norm(np.array(eye_landmarks[2]) - np.array(eye_landmarks[4]))
    C = np.linalg.norm(np.array(eye_landmarks[0]) - np.array(eye_landmarks[3]))
    EAR = (A + B) / (2.0 * C)
    return EAR

In [3]:
# Sağ ve sol göz için işaret noktalarını belirle
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

def get_eye_landmarks(face_landmarks, eye_indices):
    return [[face_landmarks.landmark[i].x, face_landmarks.landmark[i].y] for i in eye_indices]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            left_eye = get_eye_landmarks(face_landmarks, LEFT_EYE)
            right_eye = get_eye_landmarks(face_landmarks, RIGHT_EYE)

            # Sol ve sağ göz için EAR hesapla
            left_ear = calculate_eye_aspect_ratio(left_eye)
            right_ear = calculate_eye_aspect_ratio(right_eye)

            # Göz kırpma tespiti
            if left_ear < 0.2:
                print("Sol göz kırpıldı! Sandalye sola gidiyor.")
            if right_ear < 0.2:
                print("Sağ göz kırpıldı! Sandalye sağa gidiyor.")

    cv2.imshow("Yüz Jest Algılama", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break